In [1]:
from skimage import io
from skimage.color import rgb2gray
from skimage.draw import circle_perimeter
from skimage.draw import disk
from skimage.filters import gaussian
import numpy as np
from PIL import Image
from MyCannyEdgeDetector_hough import *

In [2]:
def remove_from_edge(edge,locs,R_max):
    '''
    remove previously processed edges of smaller circles to speed up the process
    '''
    H,W=edge.shape
    edge=np.pad(edge,pad_width=[[2*R_max,2*R_max],[2*R_max,2*R_max]])
    for _,h,w,_ in locs:
        rr,cc=disk((h+R_max,w+R_max),R_max)
        edge[rr,cc]=False
    return edge[2*R_max:2*R_max+H,2*R_max:2*R_max+W]

In [3]:
def my_circle_hough_transform(edge,R_min,R_max,margin_degree=30):
    R = np.arange(R_min, R_max)
    edge_points = np.argwhere(edge)
    H,W = edge.shape

    acc_array=np.zeros((len(R), H+2*R_max, W+2*R_max))
    offset=np.array([R_max, R_max])
    margin=np.cos(np.radians(90-margin_degree/2))
    for _ in range(acc_array.shape[0]):
        r=_+R_min
        for p in edge_points:
            tangent=np.array([Ix[tuple(p)],Iy[tuple(p)]])# tangent vector of point
            mag=gradient_mag[tuple(p)]
            p=p+offset
            rr, cc =circle_perimeter(p[0],p[1],r)
            circle=np.c_[rr,cc]
            orthogonal=((circle-p)*tangent).sum(axis=1)/mag/r
            indices=abs(orthogonal)<margin
            rr=rr[indices]
            cc=cc[indices]
            acc_array[_,rr,cc]+=1/r
        print('radius =',r,'processed')
    
    # normalize to between 0 and 255
    acc_array-=acc_array.min()
    acc_array/=acc_array.max()
    acc_array=acc_array*255
    return acc_array

In [4]:
def non_max_suppression(R_min,R_max,acc_array,thres,kernel_radius):
    R = np.arange(R_min, R_max)
    hh,ww=acc_array.shape[-2:]
    kernel_radius=int(kernel_radius)
    acc=acc_array.copy()
    acc[acc<thres]=0
    acc_backup=acc.copy()
    for r,h,w in np.argwhere(acc):
        r_low=0
        r_high=acc.shape[0]
        h_low=max(0,h-kernel_radius)
        h_high=min(acc.shape[1],h+kernel_radius)
        w_low=max(0,w-kernel_radius)
        w_high=min(acc.shape[2],w+kernel_radius)
        if acc[r,h,w]!=np.max(acc_backup[r_low:r_high,h_low:h_high,w_low:w_high]):
            acc[r,h,w]=0
    locs=np.argwhere(acc)
    locs[:,0]+=R_min
    # filter centers
    filtered=[]
    mask=np.zeros(acc.shape[-2:],dtype=np.bool)
    for r in R:
        centers=locs[locs[:,0]==r]
        if len(centers)==1:
            r,h,w=centers[0]
            filtered.append([r,h,w])
        elif len(centers)>1:
            mask[...]=False
            for r,h,w in centers:
                h_low=max(0,h-3)
                h_high=min(hh,h+4)
                w_low=max(0,w-3)
                w_high=min(ww,w+4)
                if mask[h_low:h_high,w_low:w_high].sum()==0:
                    mask[h,w]=True
                    filtered.append([r,h,w])
    filtered=np.array(filtered)
    output=np.c_[filtered,R_max*np.ones(filtered.shape[0],dtype=np.int)]
    return output

## read image

In [5]:
# reduce to half size
img = gaussian(rgb2gray(io.imread('images/bearing.jpg')), sigma=0.5)[::2,::2]

## find circles r=30~40

In [6]:
%%time
R_min=30
R_max=40
edge,Ix,Iy,gradient_mag = myCannyEdgeDetector(img, sigma=1, low=0.2, high=0.5)
acc_array=my_circle_hough_transform(edge,R_min,R_max)
locs3040=non_max_suppression(R_min,R_max,acc_array,thres=75,kernel_radius=2.5*R_max)
locs3040

radius = 30 processed
radius = 31 processed
radius = 32 processed
radius = 33 processed
radius = 34 processed
radius = 35 processed
radius = 36 processed
radius = 37 processed
radius = 38 processed
radius = 39 processed
CPU times: user 39.6 s, sys: 98.9 ms, total: 39.7 s
Wall time: 39.9 s


array([[ 34, 204, 108,  40],
       [ 34, 417,  24,  40],
       [ 35, 227, 574,  40],
       [ 35, 370, 213,  40],
       [ 36, 146, 295,  40],
       [ 36, 412, 630,  40],
       [ 37,  12, 648,  40],
       [ 37,  13,  77,  40],
       [ 37, 310, 400,  40]])

## find circles r=40~50

In [7]:
%%time
R_min=40
R_max=50
edge,Ix,Iy,gradient_mag = myCannyEdgeDetector(img, sigma=1.5, low=0.2, high=0.5)
edge=remove_from_edge(edge,locs=locs3040,R_max=40)
acc_array=my_circle_hough_transform(edge,R_min,R_max)
locs4050=non_max_suppression(R_min,R_max,acc_array,thres=60,kernel_radius=2*R_max)
locs4050

radius = 40 processed
radius = 41 processed
radius = 42 processed
radius = 43 processed
radius = 44 processed
radius = 45 processed
radius = 46 processed
radius = 47 processed
radius = 48 processed
radius = 49 processed
CPU times: user 33.6 s, sys: 80.1 ms, total: 33.7 s
Wall time: 33.7 s


array([[ 44,  24, 658,  50],
       [ 44, 422, 640,  50],
       [ 45, 214, 119,  50],
       [ 45, 237, 583,  50],
       [ 46, 156, 305,  50],
       [ 46, 379, 222,  50],
       [ 49,  77, 479,  50],
       [ 49, 322, 406,  50],
       [ 49, 507, 469,  50]])

## find spheres r=10~20

In [8]:
%%time
R_min=10
R_max=20
edge,Ix,Iy,gradient_mag = myCannyEdgeDetector(img, sigma=1.5, low=0.1, high=0.5)
edge=remove_from_edge(edge,locs=locs3040,R_max=40)
edge=remove_from_edge(edge,locs=locs4050,R_max=50)
acc_array=my_circle_hough_transform(edge,R_min,R_max)
locs1020=non_max_suppression(R_min,R_max,acc_array,thres=110,kernel_radius=1.5*R_max)
locs1020

radius = 10 processed
radius = 11 processed
radius = 12 processed
radius = 13 processed
radius = 14 processed
radius = 15 processed
radius = 16 processed
radius = 17 processed
radius = 18 processed
radius = 19 processed
CPU times: user 23.8 s, sys: 120 ms, total: 23.9 s
Wall time: 23.8 s


array([[ 10,  40, 107,  20],
       [ 10,  63,  52,  20],
       [ 10,  65, 507,  20],
       [ 10, 105, 475,  20],
       [ 10, 148, 561,  20],
       [ 10, 191, 286,  20],
       [ 10, 201,  24,  20],
       [ 10, 203, 150,  20],
       [ 10, 205, 488,  20],
       [ 10, 237, 488,  20],
       [ 10, 244, 115,  20],
       [ 10, 255, 509,  20],
       [ 10, 267, 437,  20],
       [ 10, 323, 435,  20],
       [ 10, 342, 571,  20],
       [ 10, 371, 131,  20],
       [ 10, 440, 567,  20],
       [ 10, 450, 377,  20],
       [ 10, 453, 626,  20],
       [ 10, 481, 284,  20],
       [ 11, 105, 419,  20],
       [ 11, 119, 207,  20],
       [ 11, 124,  93,  20],
       [ 11, 160, 329,  20],
       [ 11, 171, 227,  20],
       [ 11, 231, 395,  20],
       [ 11, 237, 607,  20],
       [ 11, 286, 309,  20],
       [ 11, 288, 178,  20],
       [ 11, 300, 232,  20],
       [ 11, 312, 312,  20],
       [ 11, 331, 618,  20],
       [ 11, 347, 337,  20],
       [ 11, 351, 256,  20],
       [ 11, 4

## find circles r=50~60

In [9]:
%%time
R_min=50
R_max=60
edge,Ix,Iy,gradient_mag = myCannyEdgeDetector(img, sigma=1, low=0.1, high=0.5)
edge=remove_from_edge(edge,locs=locs3040,R_max=40)
edge=remove_from_edge(edge,locs=locs4050,R_max=50)
acc_array=my_circle_hough_transform(edge,R_min,R_max)
locs5060=non_max_suppression(R_min,R_max,acc_array,thres=80,kernel_radius=2*R_max)
locs5060

radius = 50 processed
radius = 51 processed
radius = 52 processed
radius = 53 processed
radius = 54 processed
radius = 55 processed
radius = 56 processed
radius = 57 processed
radius = 58 processed
radius = 59 processed
CPU times: user 48.3 s, sys: 99.9 ms, total: 48.4 s
Wall time: 48.6 s


array([[ 51,  36,  97,  60],
       [ 52, 331, 419,  60],
       [ 54,  87, 491,  60],
       [ 54, 247, 590,  60],
       [ 54, 437,  43,  60],
       [ 55, 168, 312,  60],
       [ 55, 224, 127,  60],
       [ 56, 432, 647,  60],
       [ 56, 520, 474,  60],
       [ 57, 390, 232,  60]])

## find circles r=70~85

In [10]:
%%time
R_min=70
R_max=85
edge,Ix,Iy,gradient_mag = myCannyEdgeDetector(img, sigma=1.5, low=0.1, high=0.5)
edge=remove_from_edge(edge,locs=locs3040,R_max=40)
edge=remove_from_edge(edge,locs=locs4050,R_max=50)
edge=remove_from_edge(edge,locs=locs5060,R_max=60)
acc_array=my_circle_hough_transform(edge,R_min,R_max)
locs7085=non_max_suppression(R_min,R_max,acc_array,thres=80,kernel_radius=1.8*R_max)
locs7085

radius = 70 processed
radius = 71 processed
radius = 72 processed
radius = 73 processed
radius = 74 processed
radius = 75 processed
radius = 76 processed
radius = 77 processed
radius = 78 processed
radius = 79 processed
radius = 80 processed
radius = 81 processed
radius = 82 processed
radius = 83 processed
radius = 84 processed
CPU times: user 54.3 s, sys: 108 ms, total: 54.4 s
Wall time: 54.4 s


array([[ 78, 462, 675,  85],
       [ 80, 110, 513,  85],
       [ 80, 414, 258,  85],
       [ 81, 274, 619,  85],
       [ 83,  58, 695,  85],
       [ 83, 542, 506,  85],
       [ 84, 191, 339,  85],
       [ 84, 249, 154,  85],
       [ 84, 356, 448,  85],
       [ 84, 462,  69,  85]])

## find circles r=90~100

In [11]:
%%time
R_min=90
R_max=100
edge,Ix,Iy,gradient_mag = myCannyEdgeDetector(img, sigma=1.5, low=0.1, high=0.5)
edge=remove_from_edge(edge,locs=locs3040,R_max=40)
edge=remove_from_edge(edge,locs=locs4050,R_max=50)
edge=remove_from_edge(edge,locs=locs5060,R_max=60)
edge=remove_from_edge(edge,locs=locs7085,R_max=85)
acc_array=my_circle_hough_transform(edge,R_min,R_max)
locs90100=non_max_suppression(R_min,R_max,acc_array,thres=45,kernel_radius=1.5*R_max)
locs90100

radius = 90 processed
radius = 91 processed
radius = 92 processed
radius = 93 processed
radius = 94 processed
radius = 95 processed
radius = 96 processed
radius = 97 processed
radius = 98 processed
radius = 99 processed
CPU times: user 20.7 s, sys: 76 ms, total: 20.8 s
Wall time: 20.8 s


array([[ 91, 206, 353, 100],
       [ 92, 127, 528, 100],
       [ 92, 472, 690, 100],
       [ 92, 477,  87, 100],
       [ 93, 264, 171, 100],
       [ 93, 289, 632, 100],
       [ 94,  73, 138, 100],
       [ 94, 558, 519, 100],
       [ 94, 613, 331, 100],
       [ 95, 371, 459, 100],
       [ 95, 429, 273, 100],
       [ 97,  68, 713, 100],
       [ 97, 232, 817, 100]])

## postprocess
the original image is reduced to half during processing. This step convert found circle parameters back to the original.

In [12]:
locs=np.concatenate((locs1020,locs3040,locs4050,locs5060,locs7085,locs90100),axis=0)
locs[:,0]*=2
locs[:,1:3]-=locs[:,-1,None]
locs[:,1:3]*=2
print('total:', len(locs))

total: 109


## plot all

In [13]:
img_plot=io.imread('images/bearing.jpg')*0.4
H,W=img_plot.shape[:2]
img_plot=np.pad(img_plot,pad_width=[[4*R_max,4*R_max],[4*R_max,4*R_max],[0,0]])
for r,h,w,_ in locs:
    rr, cc =circle_perimeter(h,w,r)
    img_plot[rr+4*R_max,cc+4*R_max]=[0,255,0]
img_plot=img_plot.astype(np.uint8)
# show_images('', img=img_plot)

In [14]:
im = Image.fromarray(img_plot) # save padded image
im.save("images/bearing_padded_output.jpg")
im = Image.fromarray(img_plot[4*R_max:4*R_max+H,4*R_max:4*R_max+W]) # save cropped
im.save("images/bearing_cropped_output.jpg")